<div class="alert alert-warning">
    
<b>Disclaimer:</b> The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function
- setting and run the fit   
- extracting and displaying information about the results

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
</div>

# Example: Gaussian Model 3D fitted with lmfit

## Table of Contents

- [Introduction](#Introduction)
- [Importing the required libraries](#Importing-the-required-libraries)
- [Plot of the fitting model](#Plot-of-the-fitting-model)
- [Creating the reference data](#Creating-the-reference-data)
- [Setting and fitting](#Setting-and-fitting) 
- [Plotting the results](#Plotting-the-results) 

[Top](#Table-of-Contents)

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to use the <b>Gaussian Model 3D</b> model to perform some 
fits using <a href="https://lmfit.github.io/lmfit-py/">lmfit</a>.
</div>

### Physical units
For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "meV", 
                       'q': "1/Angstrom", 
                       'D': "meV.Angstrom^2", 
                       'variance_ux': "Angstrom", 
                       'scale': "unit_of_signal.meV",
                       'center': "meV"}

[Top](#Table-of-Contents)

## Importing the required librairies

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np

import matplotlib.pyplot as plt

# for interactivity (plots, buttons...)
from pandas import DataFrame
import panel.widgets as pnw
import panel as pn
pn.extension()

#%matplotlib notebook

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = pnw.Button(name='Yes', button_type='success')
    buttonN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install the QENSmodels' library?", panel.Row(buttonY, buttonN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("QENSmodels"):
    if buttonY.clicks>0:
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels
    elif buttonN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# install lmfit (if not already installed)
if not pkgutil.find_loader("lmfit"):
    lmfitY = pnw.Button(name='Yes', button_type='success')
    lmfitN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install lmfit?", panel.Row(lmfitY, lmfitN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("lmfit"):
    if lmfitY.clicks>0:
         !{sys.executable} -m pip install lmfit
    elif lmfitN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# required imports from lmfit
from lmfit import Model, Parameters

# import model from QENS library
import QENSmodels

[Top](#Table-of-Contents)

## Plot of the fitting model

The widget below shows the peak shape function imported from QENSmodels where the function's parameters can be varied.

In [ ]:
ini_values = {'q': 1., 'scale': 5., 'center': 5., 'D': 1., 'variance_ux': 1.}

# Define function to plot
def mplplot(df, **kwargs):
    fig = df.plot(legend=False).get_figure()
    plt.grid()
    plt.ylabel('Gaussian Model 3D')
    plt.xlabel('x')
    plt.close(fig)
    return fig

def gaussian_model_3d(q=1.0, scale=1.0, center=0.0, D=1., variance_ux=1.0, view_fn=mplplot):
    xs = np.linspace(-10,10,100)
    ys = QENSmodels.sqwGaussianModel3D(xs, q, scale, center, D, variance_ux) 
    df = DataFrame(dict(y=ys), index=xs)
    return view_fn(df, q=q, scale=scale, center=center, D=D, variance_ux=variance_ux)

# Define sliders and actions on plot
slider_q = pnw.FloatSlider(name='q', value=ini_values['q'], start=0.2, end=10.)
slider_scale  = pnw.FloatSlider(name='scale', value=ini_values['scale'], start=1., end=10.)
slider_center = pnw.FloatSlider(name='center', value=ini_values['center'], start=0., end=10.)
slider_D = pnw.FloatSlider(name='D', value=ini_values['D'], start=0., end=10.)
slider_variance_ux =  pnw.FloatSlider(name='variance_ux', value=ini_values['variance_ux'], start=0.01, end=10.)

def update(event):
    gaus_model_3d_panel[0] = gaussian_model_3d(slider_q.value, 
                                               slider_scale.value, 
                                               slider_center.value, 
                                               slider_D.value, slider_variance_ux.value)
    
slider_q.param.watch(update, 'value')
slider_scale.param.watch(update, 'value')
slider_center.param.watch(update, 'value')
slider_D.param.watch(update, 'value')
slider_variance_ux.param.watch(update, 'value')

# Define reset button
reset_button = pnw.Button(name='Reset')

def on_click(event):
    """Reset the interactive plots to inital values."""
    print("reset values")
    slider_q.value = ini_values['q']
    slider_scale.value = ini_values['scale']
    slider_center.value = ini_values['center']
    slider_D.value = ini_values['D']
    slider_variance_ux.value = ini_values['variance_ux']
    
reset_button.param.watch(on_click, 'clicks')

# Define layout: title, plot, sliders and reset button
widgets = pn.Column("#### Gaussian Model 3D", slider_q, slider_scale, slider_center, slider_D, slider_variance_ux, reset_button)
gaus_model_3d_panel = pn.Row(gaussian_model_3d(slider_q.value, slider_scale.value, slider_center.value, slider_D.value, slider_variance_ux.value), widgets)

gaus_model_3d_panel

[Top](#Table-of-Contents)

## Creating the reference data

In [ ]:
xx = np.linspace(-2,2,200)
gaussian_model_3d_noisy = QENSmodels.sqwGaussianModel3D(xx, .5, 0.7, 0.3, 1, 1.2)*(1+0.1*np.random.normal(0,1,200)) + 0.01*np.random.normal(0,1,200)
# plot initial mode
fig0, ax0 = plt.subplots()
ax0.plot(xx, gaussian_model_3d_noisy)
ax0.grid();

[Top](#Table-of-Contents)

## Setting and fitting

In [ ]:
gmodel = Model(QENSmodels.sqwGaussianModel3D)

print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

initial_parameters_values = [1.22, 0.25, .3, 0.33]

# Define boundaries for parameters to be refined
gmodel.set_param_hint('scale', min=0)
gmodel.set_param_hint('center', min=-5, max=5)
gmodel.set_param_hint('D', min=0)
gmodel.set_param_hint('variance_ux', min=0)

# Fix some of the parameters
gmodel.set_param_hint('q', vary=False)

# Fit
result = gmodel.fit(gaussian_model_3d_noisy, w=xx, q=1.,
                    scale=initial_parameters_values[0], 
                    center=initial_parameters_values[1],
                    D=initial_parameters_values[2],
                    variance_ux=initial_parameters_values[3])

In [ ]:
# Plot Initial model and reference data
fig1, ax1 = plt.subplots()
ax1.plot(xx, gaussian_model_3d_noisy, 'b-', label='reference data')
ax1.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
ax1.set(xlabel='x', title='Initial model and reference data')
ax1.grid()
ax1.legend();

[Top](#Table-of-Contents)

## Plotting results

using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n',result.fit_report())

# plot fitting results using lmfit functionality
result.plot();

In [ ]:
# plot fitting results and reference data using matplotlib.pyplot
fig2, ax2 = plt.subplots()
ax2.plot(xx, gaussian_model_3d_noisy, 'b-', label='reference data')
ax2.plot(xx, result.best_fit, 'r.', label='fitting result')
ax2.legend()
ax2.set(xlabel='x', title='Fit result and reference data')
ax2.grid();